In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp arc

In [ ]:
#| export
import json, os
import numpy as np
import gym
from gym import spaces
from time import sleep
import pygame
from matplotlib import colors       
# import copy

In [ ]:
#| export
import json
import numpy as np
import gym
from gym import spaces
import pygame
from matplotlib import colors
import os

class ARCEnv(gym.Env):
    def __init__(self):
        super(ARCEnv, self).__init__()
        self.index = 0
        self.env = None
        self.dimensions = []
        self._fitness = 10000  # Initialize fitness to 10000
        self.state = []
        self.done = False

        # Render settings
        self.screen_width = 1000
        self.screen_height = 500
        self.cell_size = 30
        self.left_pad = 20
        self.height_pad = 20
        self.grid_down = 50
        self.symbol_down = 150
        self.screen = None
        self.isopen = True

        self.cmap = colors.ListedColormap(['#000000', '#0074D9', '#FF4136', '#2ECC40', '#FFDC00',
                                           '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
        self.norm = colors.Normalize(vmin=0, vmax=9)

        self.fitness_label_font_size = 20
        self.fitness_value_font_size = 50

        self.iteration = 1  # Initialize iteration to 1
        self.code = ""  # Example property, ensure to set self.code elsewhere in your class
        self.data = None  # Placeholder for data

    def initialise(self, properties):
        """
        Initialize the environment with properties and data.
        """
        for key, value in properties.items():
            setattr(self, key, value)
        
        self.train_data = self.data['train']
        self.test_data = self.data['test']
        self.inputs = [x['input'] for x in self.train_data]
        self.outputs = [x.get('output', []) for x in self.train_data]
        self.reset()

    def get_train_array(self, idx):
        """
        Get the training array at the specified index.
        """
        return self.train_data[idx]

    def get_input_array(self, idx):
        """
        Get the input array at the specified index.
        """
        return self.inputs[idx]

    def get_output_array(self, idx):
        """
        Get the output array at the specified index.
        """
        return self.outputs[idx]

    def get_element(self, array, row, col):
        """
        Get the element from the array at the specified row and column.
        """
        return array[row][col]

    def get_index(self):
        """
        Get the current index.
        """
        return self.index

    def set_index(self, index):
        """
        Set the current index and reset the environment.
        """
        self.index = index
        self.reset()

    def add_rows(self, num_rows):
        """
        Add rows to the environment.
        """
        num_rows = round(num_rows)
        self.env = np.pad(self.env, ((0, num_rows), (0, 0)), mode='constant', constant_values=0)

    def remove_rows(self, num_rows):
        """
        Remove rows from the environment.
        """
        num_rows = round(num_rows)
        if num_rows >= self.env.shape[0]:
            num_rows = self.env.shape[0] - 1
        self.env = self.env[:-num_rows, :] if self.env.shape[0] > num_rows else self.env

    def add_columns(self, num_columns):
        """
        Add columns to the environment.
        """
        num_columns = round(num_columns)
        self.env = np.pad(self.env, ((0, 0), (0, num_columns)), mode='constant', constant_values=0)

    def remove_columns(self, num_columns):
        """
        Remove columns from the environment.
        """
        num_columns = round(num_columns)
        if num_columns >= self.env.shape[1]:
            num_columns = self.env.shape[1] - 1
        self.env = self.env[:, :-num_columns] if self.env.shape[1] > num_columns else self.env

    def fitness_function(self):
        """
        Calculate the fitness of the current state.
        """
        output = np.array(self.outputs[self.index])
        
        # First metric: Squared difference in dimensions
        dim_metric = (self.env.shape[0] - output.shape[0]) ** 2 + (self.env.shape[1] - output.shape[1]) ** 2
        
        # Second metric: Squared difference in elements
        element_metric = 0
        if getattr(self, 'fitness_type', 'full') != 'dim_only':
            for i in range(max(self.env.shape[0], output.shape[0])):
                for j in range(max(self.env.shape[1], output.shape[1])):
                    env_val = self.env[i, j] if i < self.env.shape[0] and j < self.env.shape[1] else None
                    output_val = output[i, j] if i < output.shape[0] and j < output.shape[1] else None
                    if env_val is None or output_val is None:
                        element_metric += 25
                    else:
                        element_metric += (env_val - output_val) ** 2

        # Final metric: Sum of the two metrics or dim_metric only
        if getattr(self, 'fitness_type', 'full') == 'dim_only':
            final_metric = dim_metric
        else:
            final_metric = dim_metric + element_metric

        return final_metric

    def step(self, action):
        """
        Take a step in the environment.
        """
        self.process_dimensions(action)
        if getattr(self, 'fitness_type', 'full') != 'dim_only':
            self.process_remaining_values(action[2:])

        self.fitness = self.fitness_function()
        
        input_shape, output_shape = self.get_env_output_dimensions()
        self.state = list(input_shape) + list(output_shape) + self.env.flatten().tolist()
        
        self.iteration += 1  # Increment iteration
        return self.state, self.fitness, self.done

    def reset(self):
        """
        Reset the environment to the initial state.
        """
        input_shape = np.array(self.inputs[self.index]).shape
        self.env = np.zeros(input_shape, dtype=int)
        self.fitness = self.fitness_function()
        self.done = False
        self.state = self.env.flatten().tolist()
        self.iteration = 1  # Reset iteration
        return self.state

    def get_env_output_dimensions(self):
        """
        Get the dimensions of the environment and output arrays.
        """
        input_shape = np.array(self.env).shape
        output_shape = np.array(self.outputs[self.index]).shape
        return input_shape, output_shape

    def process_dimensions(self, action):
        """
        Process the dimensions based on the action.
        """
        num_rows, num_cols, *values = action

        if num_rows > 0:
            self.add_rows(num_rows)
        elif num_rows < 0:
            self.remove_rows(abs(num_rows))
        
        if num_cols > 0:
            self.add_columns(num_cols)
        elif num_cols < 0:
            self.remove_columns(abs(num_cols))

    def process_remaining_values(self, values):
        """
        Process the remaining values based on the action.
        """
        for i, value in enumerate(values):
            row, col = divmod(i, self.env.shape[1])
            if row < self.env.shape[0] and col < self.env.shape[1]:
                self.env[row, col] = value

    @property
    def fitness(self):
        """
        Get the current fitness value.
        """
        return self._fitness

    @fitness.setter
    def fitness(self, value):
        """
        Set the current fitness value.
        """
        self._fitness = value

    def render(self, mode='human'):
        """
        Render the environment using Pygame.
        """
        def draw_grid(screen, grid, top_left_x, top_left_y, cell_size):
            for i, row in enumerate(grid):
                for j, value in enumerate(row):
                    if top_left_x + j * cell_size < self.screen_width and top_left_y + i * cell_size < self.screen_height:
                        color = self.cmap(self.norm(value))[:3]  # Get RGB only, excluding alpha
                        color = tuple(int(c * 255) for c in color)  # Multiply each element by 255
                        pygame.draw.rect(screen, color, (top_left_x + j * cell_size, top_left_y + i * cell_size, cell_size, cell_size))
                        pygame.draw.rect(screen, (255, 255, 255), (top_left_x + j * cell_size, top_left_y + i * cell_size, cell_size, cell_size), 1)
            # Draw a black line around the grid
            if top_left_x < self.screen_width and top_left_y < self.screen_height:
                pygame.draw.rect(screen, (0, 0, 0), (top_left_x, top_left_y, cell_size * len(grid[0]), cell_size * len(grid)), 2)

        if self.screen is None:
            pygame.init()
            self.screen = pygame.display.set_mode((self.screen_width, self.screen_height))
            pygame.display.set_caption('ARC Environment')

        input_grid = np.array(self.inputs[self.index])
        output_grid = np.array(self.outputs[self.index])
        env_grid = self.env

        # Top left coordinates
        input_grid_x = self.left_pad
        input_grid_y = self.grid_down
        arrow_img_x = input_grid_x + input_grid.shape[1] * self.cell_size + self.left_pad
        arrow_img_y = self.symbol_down
        output_grid_x = arrow_img_x + 50 + self.left_pad
        output_grid_y = self.grid_down
        equals_img_x = output_grid_x + output_grid.shape[1] * self.cell_size + self.left_pad
        equals_img_y = self.symbol_down
        env_grid_x = equals_img_x + 50 + self.left_pad
        env_grid_y = self.grid_down
        fitness_text_x = env_grid_x + env_grid.shape[1] * self.cell_size + self.left_pad
        fitness_text_y = self.grid_down + self.height_pad
        fitness_value_y = fitness_text_y + self.fitness_label_font_size + self.height_pad
        tick_cross_y = fitness_value_y + self.fitness_value_font_size + self.height_pad
        table_y = self.grid_down + max(input_grid.shape[0], output_grid.shape[0], env_grid.shape[0]) * self.cell_size + self.height_pad

        # Adjust screen size if necessary
        if fitness_text_x + self.fitness_value_font_size > self.screen_width:
            self.screen_width = fitness_text_x + self.fitness_value_font_size + self.left_pad
            self.screen = pygame.display.set_mode((self.screen_width, self.screen_height))
        if env_grid_y + env_grid.shape[0] * self.cell_size > self.screen_height:
            self.screen_height = env_grid_y + env_grid.shape[0] * self.cell_size + self.height_pad
            self.screen = pygame.display.set_mode((self.screen_width, self.screen_height))

        self.screen.fill((255, 255, 255))  # Clear the screen

        draw_grid(self.screen, input_grid, input_grid_x, input_grid_y, self.cell_size)
        draw_grid(self.screen, output_grid, output_grid_x, output_grid_y, self.cell_size)
        draw_grid(self.screen, env_grid, env_grid_x, env_grid_y, self.cell_size)

        # Load and scale images
        arrow_img = pygame.transform.scale(pygame.image.load('images/arrow.png'), (50, 50))
        equals_img = pygame.transform.scale(pygame.image.load('images/equals.jpg'), (50, 50))
        green_tick_img = pygame.transform.scale(pygame.image.load('images/green_tick.png'), (50, 50))
        red_cross_img = pygame.transform.scale(pygame.image.load('images/red-cross.png'), (50, 50))

        if arrow_img_x < self.screen_width and arrow_img_y < self.screen_height:
            self.screen.blit(arrow_img, (arrow_img_x, arrow_img_y))
        if equals_img_x < self.screen_width and equals_img_y < self.screen_height:
            self.screen.blit(equals_img, (equals_img_x, equals_img_y))

        # Display fitness text
        label_font = pygame.font.Font(None, self.fitness_label_font_size)
        value_font = pygame.font.Font(None, self.fitness_value_font_size)
        label_font.set_bold(True)
        fitness_label = label_font.render("Fitness:", True, (0, 0, 0))
        fitness_value = value_font.render(f"{self.fitness:.2f}", True, (0, 0, 0))
        if fitness_text_x < self.screen_width and fitness_text_y < self.screen_height:
            self.screen.blit(fitness_label, (fitness_text_x, fitness_text_y))
        if fitness_text_x < self.screen_width and fitness_value_y < self.screen_height:
            self.screen.blit(fitness_value, (fitness_text_x, fitness_value_y))

        if self.fitness < 1e-6:
            if fitness_text_x < self.screen_width and tick_cross_y < self.screen_height:
                self.screen.blit(green_tick_img, (fitness_text_x, tick_cross_y))
        else:
            if fitness_text_x < self.screen_width and tick_cross_y < self.screen_height:
                self.screen.blit(red_cross_img, (fitness_text_x, tick_cross_y))

        # Draw table with Code and Iteration
        table_font = pygame.font.Font(None, 24)
        table_labels = ["Code", "Iteration"]
        table_values = [os.path.splitext(self.code)[0], self.iteration]  # Display code without file extension
        table_x = self.left_pad
        table_y = table_y

        for i, (label, value) in enumerate(zip(table_labels, table_values)):
            label_surface = table_font.render(label, True, (0, 0, 0))
            value_surface = table_font.render(str(value), True, (0, 0, 0))
            self.screen.blit(label_surface, (table_x + i * 200, table_y))
            self.screen.blit(value_surface, (table_x + i * 200, table_y + 30))

        pygame.display.flip()

    def close(self):
        """
        Close the environment, save the screen to an HTML and an image file.
        """
        if self.screen is not None:
            pygame.image.save(self.screen, "screen_image.png")
            
            # Save the screen image to an HTML format
            with open("screen_image.html", "w") as f:
                f.write(f"<html><body><img src='screen_image.png'></body></html>")

            pygame.display.quit()
            pygame.quit()
            self.isopen = False



In [ ]:
#| gui
# Example usage:
props = {'dir': 'C:\\packages\\arc-prize-2024\\training', 'code':'1_007bbfb7.dat', 'fitness_type': 'dim_only'}
file_name = os.path.join(props['dir'], props['code'])
with open(file_name, 'r') as f:
    data = json.load(f)

properties={}
properties['data'] = data
arc_env = ARCEnv()
arc_env.initialise(properties)
arc_env.render()
#    print(state, fitness, done)
print(arc_env.dimensions)
for i in range(6):
    state, fitness, done = arc_env.step([1, 1, 1, 2, 3, 4, 5, 6, 7])
    # print(state, fitness, done)
    print(arc_env.dimensions, fitness, done)
    arc_env.render()
    sleep(2)



[]
[] 1962 False
[] 2139 False
[] 2256 False
[] 2166 False
[] 2373 False
[] 2534 False


In [ ]:
#|gui

sleep(5)
arc_env.close()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()